In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
from pymongo import MongoClient
import ystockquote

In [3]:
from datetime import datetime, timedelta, timezone
import pytz
import math

# Load transcripts

In [4]:
est_tz = pytz.timezone('US/Eastern')
def utc_to_est(utc_dt):
    local_dt = utc_dt.replace(tzinfo=pytz.utc).astimezone(est_tz)
    return est_tz.normalize(local_dt)

In [5]:
def shift_time_according_to_closing_time(dt):
    if dt.hour >= 16:
        return dt + timedelta(days=1)
    return dt

In [6]:
def load_and_clean_transcripts():
    client = MongoClient('localhost', 27017)
    earnings_transcript_collection = client.python_import.earnings_transcript
    
    earnings_transcript = pd.DataFrame(list(earnings_transcript_collection.find()))
    earnings_transcript.drop('_id', axis=1, inplace=True)
    earnings_transcript['shifted_publishDate'] = earnings_transcript.apply(
        lambda row: datetime.strptime(row['publishDate'], '%Y-%m-%dT%H:%M:%SZ'), axis=1
    )
    earnings_transcript['shifted_publishDate'] = earnings_transcript.apply(
        lambda row: utc_to_est(row['shifted_publishDate']), axis=1
    )
    earnings_transcript['shifted_publishDate'] = earnings_transcript.apply(
        lambda row: shift_time_according_to_closing_time(row['shifted_publishDate']), axis=1
    )
    earnings_transcript['publishDate_str'] = earnings_transcript.apply(
        lambda row: str(row['shifted_publishDate'].date()), axis=1
    )
    earnings_transcript.set_index(['tradingSymbol', 'publishDate_str'], inplace=True)
    
    return earnings_transcript

In [7]:
earnings_transcript = load_and_clean_transcripts()

In [8]:
earnings_transcript.head(2)

,,publishDate,qAndAText,rawText,url,shifted_publishDate
tradingSymbol,publishDate_str,,,,,
A,2014-11-18,2014-11-17T22:25:00Z,,The following audio is from a conference call ...,https://seekingalpha.com/article/2675895-agile...,2014-11-18 17:25:00-05:00
AA,2014-10-09,2014-10-09T01:18:03Z,Operator [Operator instructions.] Our first qu...,Alcoa (NYSE: AA ) Q3 2014 Results Earnings Con...,https://seekingalpha.com/article/2549515-alcoa...,2014-10-09 21:18:03-04:00


In [9]:
earnings_transcript.tail(2)

publishDate  \
tradingSymbol publishDate_str                         
DLPH          2017-02-02       2017-02-02T14:17:56Z   
              2017-02-02       2017-02-02T19:57:18Z   

                                                                       qAndAText  \
tradingSymbol publishDate_str                                                      
DLPH          2017-02-02                                                           
              2017-02-02       Operator And our first question comes from the...   

                                                                         rawText  \
tradingSymbol publishDate_str                                                      
DLPH          2017-02-02       The following slide deck was published by Delp...   
              2017-02-02       Delphi Automotive Plc (NYSE: DLPH ) Q4 2016 Ea...   

                                                                             url  \
tradingSymbol publishDate_str                                                      
DLPH          2017-02-02       https://seekingalpha.com/article/4041924-delph...   
              2017-02-02       https://seekingalpha.com/article/4042152-delph...   

                                    shifted_publishDate  
tradingSymbol publishDate_str                            
DLPH          2017-02-02      2017-02-02 09:17:56-05:00  
              2017-02-02      2017-02-02 14:57:18-05:00

### Load all ticker data for transcripts

In [10]:
def construct_dataframe_from_stock_data(arr, ticker):
    df = pd.DataFrame(arr).transpose()
    if 'Adj Close' in df.columns:
        df['Close'] = pd.to_numeric(df['Adj Close'], errors='ignore')
    elif 'Close' in df.columns:
        df['Close'] = pd.to_numeric(df['Close'], errors='ignore')
    else:
        print('{} does not have close prices'.format(ticker))
        
    df.drop(axis=1, labels=[col for col in df.columns if col not in ['Close']], inplace=True)
    indexes = pd.MultiIndex.from_product([[ticker], df.index.values.tolist()], names=['tradingSymbol', 'publishDate_str'])
    df.set_index(indexes, inplace=True)
    
    return df

In [11]:
def calc_labels(df):
    df['Std Dev'] = pd.Series(data=df['Close']).rolling(window=20,center=False).std()
    df['1day return'] = df['Close'].shift(-1) - df['Close']
    df['1day pct change'] = df['1day return']/df['Close']*100
    df['5day return'] = df['Close'].shift(-5) - df['Close']
    df['5day pct change'] = df['5day return']/df['Close']*100

In [12]:
SPY = ystockquote.get_historical_prices('SPY', '2000-01-01', '2017-12-31')

In [13]:
SPY_df = construct_dataframe_from_stock_data(SPY, 'SPY')

In [14]:
SPY_df.head(5)

Close
tradingSymbol publishDate_str            
SPY           2000-01-03       105.366938
              2000-01-04       101.246443
              2000-01-05       101.427563
              2000-01-06        99.797478
              2000-01-07       105.593338

In [15]:
calc_labels(SPY_df)
SPY_df.dropna(inplace=True)

In [16]:
SPY_df.head()

Close   Std Dev  1day return  \
tradingSymbol publishDate_str                                      
SPY           2000-01-31       101.110602  2.375208     0.996164   
              2000-02-01       102.106766  2.346908     0.090560   
              2000-02-02       102.197326  2.313233     1.539526   
              2000-02-03       103.736852  2.271400    -0.430195   
              2000-02-04       103.306657  2.105601    -0.158447   

                               1day pct change  5day return  5day pct change  
tradingSymbol publishDate_str                                                 
SPY           2000-01-31              0.985222     2.037608         2.015227  
              2000-02-01              0.088691     2.445129         2.394679  
              2000-02-02              1.506425     0.158448         0.155041  
              2000-02-03             -0.414698    -1.177285        -1.134876  
              2000-02-04             -0.153375    -2.829977        -2.739395

In [17]:
tickers = earnings_transcript.index.levels[0]
all_stocks = None
try:
    all_stocks = pd.read_pickle('all_stocks.pkl')
except:
    pass

print('Downloading missing stock data')

for ticker in tickers:
    if (all_stocks is not None) and (ticker in all_stocks.index):
        continue
    try:
        stocks = ystockquote.get_historical_prices(ticker, '2000-01-01', '2017-12-31')
    except:
        print('{} is not available on Yahoo'.format(ticker))
        continue
    print('{} ticker has {} long list'.format(ticker, len(stocks)))
    
    df = construct_dataframe_from_stock_data(stocks, ticker)
    calc_labels(df)
    df.dropna(inplace=True)
    
    SPY_df.index.set_levels([[ticker], SPY_df.index.levels[1]], inplace=True)
    
    df['1day relative pct change'] = df['1day pct change'] - SPY_df['1day pct change']
    df['5day relative pct change'] = df['5day pct change'] - SPY_df['5day pct change']
    
    if all_stocks is None:
        all_stocks = df
    else:
        all_stocks = all_stocks.append(df)
        
all_stocks.to_pickle('all_stocks.pkl')

A ticker has 4349 long list
AA ticker has 4349 long list
AAN ticker has 4349 long list
AAON ticker has 4349 long list
AAP ticker has 3871 long list
AAPL ticker has 4349 long list
AAT ticker has 1574 long list
AAWW ticker has 2877 long list
ABAX ticker has 4349 long list
ABBV ticker has 1080 long list
ABC ticker has 4349 long list
ABCB ticker has 4349 long list
ABG ticker has 3795 long list
ABM ticker has 4349 long list
ABT ticker has 4349 long list
ACAT ticker has 4320 long list
ACC ticker has 3190 long list
ACE is not available on Yahoo
ACET ticker has 4349 long list
ACIW ticker has 4349 long list
ACM ticker has 2502 long list
ACN ticker has 3960 long list
ACOR ticker has 2814 long list
ACT is not available on Yahoo
ACXM ticker has 4349 long list
ADBE ticker has 4349 long list
ADC ticker has 4349 long list
ADI ticker has 4349 long list
ADM ticker has 4349 long list
ADNT ticker has 124 long list
ADP ticker has 4349 long list
ADPT ticker has 708 long list
ADS ticker has 3983 long list
A

In [26]:
all_stocks.sample(20)

,,Close,Std Dev,1day return,1day pct change,5day return,5day pct change,1day relative pct change,5day relative pct change
tradingSymbol,publishDate_str,,,,,,,,
AGCO,2006-11-02,27.076669,1.075552,-0.163634,-0.604336,2.185001,8.069682,-0.428868,7.046145
CY,2008-01-30,3.680734,0.898391,-0.001740,-0.047273,-0.187070,-5.082410,-1.870705,-3.703713
AA,2014-12-24,36.984048,2.273917,0.117262,0.317061,0.257975,0.697530,-0.005410,1.823782
AKR,2004-11-08,8.974366,0.152904,0.029716,0.331121,0.059433,0.662253,0.527520,-1.021525
APOG,2002-02-12,8.974893,0.854092,0.045481,0.506758,-0.197084,-2.195948,-0.555444,-1.745862
ANGO,2010-08-06,16.000000,0.340193,0.469999,2.937494,-0.190000,-1.187500,2.403640,2.442718
AMZN,2012-02-03,187.679993,6.342532,-4.539994,-2.419008,-2.140000,-1.140239,-2.352116,-1.006455
ATW,2000-08-09,9.791444,0.438681,-0.163196,-1.666720,0.311541,3.181768,-1.179194,2.376342
DECK,2012-08-23,49.049999,3.459024,0.490002,0.998985,-0.029999,-0.061160,0.394697,0.059697


In [19]:
all_stocks.tail(5)

Close   Std Dev  1day return  \
tradingSymbol publishDate_str                                     
DLPH          2017-04-03       78.250000  1.963170    -2.269997   
              2017-04-04       75.980003  1.962236    -0.890007   
              2017-04-05       75.089996  2.104154     0.030007   
              2017-04-06       75.120003  2.208314     0.019996   
              2017-04-07       75.139999  2.350559     0.390000   

                               1day pct change  5day return  5day pct change  \
tradingSymbol publishDate_str                                                  
DLPH          2017-04-03             -2.900955    -2.720001        -3.476040   
              2017-04-04             -1.171370    -1.069999        -1.408264   
              2017-04-05              0.039961    -0.809997        -1.078702   
              2017-04-06              0.026619    -1.370003        -1.823753   
              2017-04-07              0.519031    -0.430000        -0.572265   

                               1day relative pct change  \
tradingSymbol publishDate_str                             
DLPH          2017-04-03                      -2.964692   
              2017-04-04                      -0.874106   
              2017-04-05                      -0.241154   
              2017-04-06                       0.128558   
              2017-04-07                       0.459508   

                               5day relative pct change  
tradingSymbol publishDate_str                            
DLPH          2017-04-03                      -3.480286  
              2017-04-04                      -1.229906  
              2017-04-05                      -0.759254  
              2017-04-06                      -0.579271  
              2017-04-07                      -0.304412

# Merge transcripts with stock data

In [20]:
merged_data = earnings_transcript.merge(all_stocks, left_index=True, right_index=True)

In [21]:
merged_data.sample(5)

,,publishDate,qAndAText,rawText,url,shifted_publishDate,Close,Std Dev,1day return,1day pct change,5day return,5day pct change,1day relative pct change,5day relative pct change
tradingSymbol,publishDate_str,,,,,,,,,,,,,
APA,2011-11-04,2011-11-04T01:30:12Z,Operator [Operator Instructions] Your first qu...,Apache (NYSE: APA ) Q3 2011 Earnings Call Nove...,https://seekingalpha.com/article/305203-apache...,2011-11-04 21:30:12-04:00,93.375693,5.321505,1.183508,1.267469,4.202840,4.501000,0.645858,3.560611
DHI,2014-07-24,2014-07-24T13:55:00Z,,The following audio is from a conference call ...,https://seekingalpha.com/article/2336945-q3-20...,2014-07-24 09:55:00-04:00,21.286876,0.635615,-0.320176,-1.504100,-1.203086,-5.651773,-1.035944,-2.852882
ATI,2016-10-25,2016-10-25T17:43:12Z,Operator Sure. We will now begin the question-...,"Allegheny Technologies, Inc. (NYSE: ATI ) Q3 2...",https://seekingalpha.com/article/4014658-alleg...,2016-10-25 13:43:12-04:00,15.120000,0.734599,-0.850000,-5.621693,-1.750000,-11.574074,-5.420922,-10.098609
AKAM,2017-03-31,2017-03-31T17:04:36Z,,The following slide deck was published by Akam...,https://seekingalpha.com/article/4059499-akama...,2017-03-31 13:04:36-04:00,59.700001,1.307269,-1.600003,-2.680072,-1.600003,-2.680072,-2.506150,-2.451003
ADI,2006-02-16,2006-02-16T18:12:15Z,,Analog Devices Inc. (NYSE: ADI ) Q1 2006 Ear...,https://seekingalpha.com/article/6884-analog-d...,2006-02-16 13:12:15-05:00,29.059107,0.628357,-0.580009,-1.995963,-0.806209,-2.774376,-1.724977,-2.967935


In [22]:
merged_data = merged_data[merged_data['rawText'].apply(len) > 5000]

In [23]:
merged_data.to_pickle('merged_data_pct_change.pkl')